# Procesamiento del Lenguaje 

1. Darwin Yusef Gonzalez 
2. Orlando Silva 
3. Jose Barrios
4. Ana Hernandez
June 2025

## Laboratorio de Aprendizaje supervisado basado en desambiguar el sentido (senseval) de las palabras
En este laboratorio Hemos implementado el uso de diferentes algoritmos basados en aprendizaje automático supervisado para desambiguar el sentido de las palabras en Python y utilizando la herramienta de software abierto Natural Language Toolkit (`NLTK`).

# Instalación y configuraciónes iniciales

In [347]:
# 🤓 Iniciamos con la configuración y extracción de las librerías senseval, punk stopwords wordnet,
import os
import nltk
from nltk.corpus import senseval
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import string
from collections import Counter
import re
import string

nltk.download('senseval') # Para el corpus Senseval 2
nltk.download('punkt')    # Para tokenización
nltk.download('stopwords') # Para palabras vacías (stop words)
nltk.download('wordnet')  # Aunque no se use directamente para los sentidos, es útil para entender WordNet.
nltk.download('averaged_perceptron_tagger') # Para etiquetado POS si se necesita, aunque Senseval ya viene con ello.

[nltk_data] Downloading package senseval to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package senseval is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Aqui
[nltk_data]     Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Aqui Creamos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [348]:
# 🤓 Listamos todas las instancias
senseval.fileids()

['hard.pos', 'interest.pos', 'line.pos', 'serve.pos']

In [349]:
hard_instances = senseval.instances('hard.pos')
hard_instances = [instance for instance in hard_instances if instance.senses]  # Filtrar instancias con sentidos
print(f"Total de instancias con sentidos: {len(hard_instances)}")

Total de instancias con sentidos: 4333


In [350]:
file_path = 'corpus_etiquetado.txt'

# 🤓 Cargamos el archivo de texto que contiene el corpus
try:
    with open(file_path, 'r', encoding='utf-8') as f:
        corpus_content = f.read()
    print(f"Archivo '{file_path}' cargado exitosamente.")
except FileNotFoundError:
    print(f"Error: El archivo '{file_path}' no se encontró. Asegúrate de que está en la misma carpeta que tu script.")
except Exception as e:
    print(f"Ocurrió un error al leer el archivo: {e}")

Archivo 'corpus_etiquetado.txt' cargado exitosamente.


In [351]:
## 🤓Procesa el contenido del corpus para extraer pares (palabra, etiqueta).
## 🤓 Maneja etiquetas de documento, líneas vacías y el formato específico.
def parse_corpus_content(content):
    corpus = []
    
    for linea in content.splitlines():
        linea = linea.strip()
        
        # Saltamos etiquetas de documento y líneas vacías.
        if linea.startswith("<doc") or linea.startswith("</doc>") or linea == "":
            continue
        
        # Saltar 'Fp' si aparece en una línea separada (inconsistencia del corpus).
        if linea == "Fp":
            continue
        
        datos = linea.split("\t")
        
        # Formato esperado: palabra \t lema \t etiqueta
        if len(datos) == 3:
            word = datos[0]
            tag = datos[2]
            corpus.append((word, tag))
        # Caso específico: . \t . \t Fp
        elif len(datos) == 2 and datos[0] == '.' and datos[1] == 'Fp':
            word = datos[0]
            tag = datos[1] 
            corpus.append((word, tag))
        # Las líneas que no encajan se omiten.
        else:
            pass 

    return corpus


# Analisis del codigo inicial y uso, levantamiento del corpus

In [352]:
parse_corpus_content(corpus_content) # Procesamos el contenido del corpus de manera organizada
# 🤓 Todo ello para extraer pares (palabra, etiqueta)

[('Habla', 'VMIP3S0'),
 ('con', 'SP'),
 ('el', 'DA'),
 ('enfermo', 'NCMS000'),
 ('grave', 'AQ0CS00'),
 ('de', 'SP'),
 ('trasplantes', 'NCMN000'),
 ('.', 'Fp'),
 ('El', 'DA'),
 ('enfermo', 'NCMS000'),
 ('grave', 'AQ0CS00'),
 ('habla', 'VMIP3S0'),
 ('de', 'SP'),
 ('trasplantes', 'NCMN000'),
 ('.', 'Fp'),
 ('La', 'DA'),
 ('película', 'NCFS000'),
 ('fue', 'VAIP3S0'),
 ('nominada', 'VMP00SF'),
 ('al', 'SP+DA'),
 ('Oscar', 'NP00000'),
 ('.', 'Fp'),
 ('Luis', 'NP00000'),
 ('Buñuel', 'NP00000'),
 ('es', 'VSIP3S0'),
 ('director', 'NCMS000'),
 ('español', 'AQ0MS00'),
 ('.', 'Fp'),
 ('El', 'DA'),
 ('niño', 'NCMS000'),
 ('corre', 'VMIP3S0'),
 ('en', 'SP'),
 ('el', 'DA'),
 ('parque', 'NCMS000'),
 ('.', 'Fp'),
 ('Una', 'DI'),
 ('flor', 'NCFS000'),
 ('roja', 'AQ0FS00'),
 ('decoraba', 'VIIIS0S'),
 ('la', 'DA'),
 ('mesa', 'NCFS000'),
 ('.', 'Fp'),
 ('Ella', 'PP3FS000'),
 ('canta', 'VMIP3S0'),
 ('una', 'DI'),
 ('canción', 'NCFS000'),
 ('alegre', 'AQ0CS00'),
 ('.', 'Fp'),
 ('Ellos', 'PP3MP000'),
 ('juega

In [353]:
corpus = parse_corpus_content(corpus_content)
print("Corpus cargado. Número de tokens:", len(corpus))
print("Primeros 5 registros:", corpus[:5])

# 🤓 Contamos las etiquetas y mostramos las más frecuentes, por eso filtramos :5.

Corpus cargado. Número de tokens: 137
Primeros 5 registros: [('Habla', 'VMIP3S0'), ('con', 'SP'), ('el', 'DA'), ('enfermo', 'NCMS000'), ('grave', 'AQ0CS00')]


In [354]:
# 🤓 cargamos esats instancias hay que tener en cuenta que en google colab nos genero bastantes problemas
hard_instances = []
try:
    # Instanciamos a nivel de palabras tipo "hard"
    print("Cargando instancias de 'hard' desde Senseval...")
    hard_instances = senseval.instances('hard.pos')
    # Filter instances to ensure they have senses, as some might not
    hard_instances = [instance for instance in hard_instances if instance.senses]
    print(f"Total Senseval 'hard' instances with senses: {len(hard_instances)}")
except Exception as e:
    print(f"Could not load 'hard.pos' from senseval, skipping 'hard' analysis for senses: {e}")


Cargando instancias de 'hard' desde Senseval...
Total Senseval 'hard' instances with senses: 4333


In [355]:
# 🤓 Inicializamos las instancias de "serve" de Senseval
serve_instances = []
try:
    # 🤓 Cargamos las instancias para "serve" desde Senseval
    serve_instances = senseval.instances('serve.pos')
    # Filtrar instancias para asegurarse de que tienen sentidos, ya que algunas podrían no tenerlos
    serve_instances = [instance for instance in serve_instances if instance.senses]
    print(f"Total Senseval 'serve' instances with senses: {len(serve_instances)}")
except Exception as e:
    print(f"Could not load 'serve.pos' from senseval, skipping 'serve' analysis for senses: {e}")



Total Senseval 'serve' instances with senses: 4378


1. ¿Cuántos posibles sentidos tienen las palabras ambiguas «hard» y «serve»? ¿Cuáles son esos sentidos? Para cada sentido indica la etiqueta que aparece en el corpus.

In [356]:
print("--- 1. Posibles sentidos y etiquetas para 'hard and 'serve' ---")

hard_senses = set()
for instance in hard_instances:
    for sense in instance.senses:
        hard_senses.add(sense)
print(f"'hard' has {len(hard_senses)} possible senses: {hard_senses}")

serve_senses = set()
for instance in serve_instances:
    for sense in instance.senses:
        serve_senses.add(sense)
print(f"'serve' has {len(serve_senses)} possible senses: {serve_senses}")



--- 1. Posibles sentidos y etiquetas para 'hard and 'serve' ---
'hard' has 3 possible senses: {'HARD2', 'HARD1', 'HARD3'}
'serve' has 4 possible senses: {'SERVE6', 'SERVE10', 'SERVE2', 'SERVE12'}


¿Cuántas instancias hay en el corpus para cada uno de los sentidos de las palabras ambiguas «hard» y «serve»? Es decir, cuantas oraciones hay en el corpus etiquetadas con cada uno de los sentidos.

In [357]:
print("\n--- 2. Numero de instancias para cada sentido ---")
# --- 2. Número de instancias para cada sentido ---
# Contamos las instancias para cada sentido de la palabra ambigua 'hard'
hard_sense_counts = {}
for instance in hard_instances:
    for sense in instance.senses:
        hard_sense_counts[sense] = hard_sense_counts.get(sense, 0) + 1
print("\nConteo de sentidos para 'hard':")
for sense, count in hard_sense_counts.items():
    print(f"  Sentido '{sense}': {count} instancias")

# Contamos las instancias para cada sentido de la palabra ambigua 'serve'
serve_sense_counts = {}
for instance in serve_instances:
    for sense in instance.senses:
        serve_sense_counts[sense] = serve_sense_counts.get(sense, 0) + 1
print("\nConteo de sentidos para 'serve':")
for sense, count in serve_sense_counts.items():
    print(f"  Sentido '{sense}': {count} instancias")




--- 2. Numero de instancias para cada sentido ---

Conteo de sentidos para 'hard':
  Sentido 'HARD1': 3455 instancias
  Sentido 'HARD2': 502 instancias
  Sentido 'HARD3': 376 instancias

Conteo de sentidos para 'serve':
  Sentido 'SERVE10': 1814 instancias
  Sentido 'SERVE12': 1272 instancias
  Sentido 'SERVE2': 853 instancias
  Sentido 'SERVE6': 439 instancias


3. ¿Tienen todas las instancias que forman el corpus el formato que se ha descrito anteriormente? Si hay alguna instancia que no cumpla con ese formato, indica cuales serían las incongruencias que presenta y muestra algunos ejemplos.

In [358]:
# 🤓 3. Formatos gramaticales para "hard" y "serve"
print("--- 3. Formatos gramaticales para 'hard' y 'serve' ---")

# 🤓 Para 'hard'
hard_forms = set()
for instance in hard_instances:
    # 🤓 instance.context puede contener tuplas (palabra, etiqueta) o solo palabras.
    # 🤓 Intentaremos manejar ambos casos verificando el tipo de 'item'.
    for item in instance.context:
        word_to_check = ''
        if isinstance(item, tuple):
            word_to_check = item[0]  # 🤓 Obtener la palabra de la tupla
        elif isinstance(item, str):
            word_to_check = item  # 🤓 'item' ya es la palabra como cadena
        
        if word_to_check:  # 🤓 Asegurarse de que no esté vacío
            if word_to_check.lower() in {'hard', 'harder', 'hardest'}:
                hard_forms.add(word_to_check.lower())
print(f"Formatos gramaticales en 'hard' encontrados: {hard_forms}")

# 🤓 Para 'serve' 
serve_forms = set()
for instance in serve_instances:
    for item in instance.context:
        word_to_check = ''
        if isinstance(item, tuple):
            word_to_check = item[0]
        elif isinstance(item, str):
            word_to_check = item
            
        if word_to_check:
            if word_to_check.lower().startswith('serve'):  # 🤓 Capturar serve, serves, served, serving, etc.
                serve_forms.add(word_to_check.lower())
print(f"Formatos gramaticales en 'serve' encontrados: {serve_forms}")

--- 3. Formatos gramaticales para 'hard' y 'serve' ---
Formatos gramaticales en 'hard' encontrados: {'hardest', 'harder', 'hard'}
Formatos gramaticales en 'serve' encontrados: {'serves', 'server', 'serve', 'served'}


In [359]:
inst = hard_instances[1]
print(inst)

SensevalInstance(word='hard-a', position=10, context=[('clever', 'NNP'), ('white', 'NNP'), ('house', 'NNP'), ('``', '``'), ('spin', 'VB'), ('doctors', 'NNS'), ("''", "''"), ('are', 'VBP'), ('having', 'VBG'), ('a', 'DT'), ('hard', 'JJ'), ('time', 'NN'), ('helping', 'VBG'), ('president', 'NNP'), ('bush', 'NNP'), ('explain', 'VB'), ('away', 'RB'), ('the', 'DT'), ('economic', 'JJ'), ('bashing', 'NN'), ('that', 'IN'), ('low-and', 'JJ'), ('middle-income', 'JJ'), ('workers', 'NNS'), ('are', 'VBP'), ('taking', 'VBG'), ('these', 'DT'), ('days', 'NNS'), ('.', '.')], senses=('HARD1',))


4. ¿Tienen todas las instancias que forman el corpus el formato que se ha descrito anteriormente? Si hay alguna instancia que no cumpla con ese formato, indica cuales serían las incongruencias que presenta y muestra algunos ejemplos.


In [360]:

# Check if all instances in the provided corpus follow the format
print("\n--- 4. Checking corpus_etiquetado.txt format consistency ---")

if not inconsistent_lines:
    print("Todas las inconsistencias en el modulo 'corpus_etiquetado.txt' que aparecen el  formato (word\\tlemma\\ttag).")
else:
    print("Por si se encuentra alguna inconsistencia 'corpus_etiquetado.txt':")
    for i, problem in enumerate(inconsistent_lines[:10]): # Show first 10 examples
        print(f"  {i+1}. {problem}")
    if len(inconsistent_lines) > 10:
        print(f"  ...and {len(inconsistent_lines) - 10} more inconsistencies.")

# Example of a well-formatted line from your corpus for reference
print("\nEjemplo como lo pide la guia con (word\\tlemma\\ttag):")
print("Habla\\thablar\\tVMIP3S0")
print("con\\tcon\\tSP")
print("el\\tel\\tDA")


--- 4. Checking corpus_etiquetado.txt format consistency ---
Por si se encuentra alguna inconsistencia 'corpus_etiquetado.txt':
  1. Invalid tag format: .	.	Fp (Doc ID: 1)
  2. Invalid tag format: .	.	Fp (Doc ID: 2)
  3. Invalid tag format: .	.	Fp (Doc ID: 3)
  4. Invalid tag format: .	.	Fp (Doc ID: 4)
  5. Invalid tag format: .	.	Fp (Doc ID: 5)
  6. Invalid tag format: .	.	Fp (Doc ID: 6)
  7. Invalid tag format: .	.	Fp (Doc ID: 7)
  8. Invalid tag format: .	.	Fp (Doc ID: 8)
  9. Invalid tag format: .	.	Fp (Doc ID: 9)
  10. Invalid tag format: .	.	Fp (Doc ID: 10)
  ...and 10 more inconsistencies.

Ejemplo como lo pide la guia con (word\tlemma\ttag):
Habla\thablar\tVMIP3S0
con\tcon\tSP
el\tel\tDA


In [361]:
def verify_corpus_format(file_path=corpus_content):
    
    if not os.path.exists(file_path):
        print(f"Error: The file '{file_path}' was not found.")
        return

    print(f"--- Verifying format of '{file_path}' ---")

    current_doc_id = None
    line_number = 0
    inconsistencies = []
    example_lines = []
    doc_count = 0

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            line_number += 1
            line = line.strip() # Remove leading/trailing whitespace including newline

            if not line: # Empty line, often signifies end of a doc or just whitespace
                continue

            if line.startswith('<doc id="') and line.endswith('>'):
                doc_count += 1
                try:
                    # Extract doc ID
                    current_doc_id = line.split('"')[1]
                    # print(f"\n--- Processing Document ID: {current_doc_id} ---")
                except IndexError:
                    inconsistencies.append(f"Line {line_number}: Malformed document ID tag: '{line}'")
                continue

            # If it's not a doc ID line and not empty, it should be a data line
            parts = line.split('\t')

            if len(parts) == 3:
                # Valid format: Word<TAB>Lemma<TAB>POS_Tag
                if len(example_lines) < 3 and current_doc_id is not None: # Capture up to 3 valid examples
                    example_lines.append(f"Doc {current_doc_id}, Line {line_number}: '{line}' (Valid)")
            else:
                # Inconsistent format
                inconsistencies.append(f"Doc {current_doc_id if current_doc_id else 'N/A'}, Line {line_number}: Expected 3 tab-separated fields, found {len(parts)} in '{line}'")

    print(f"\n--- Summary for '{file_path}' ---")
    print(f"Total documents processed: {doc_count}")

    if inconsistencies:
        print(f"\nFound {len(inconsistencies)} inconsistencies:")
        for i, issue in enumerate(inconsistencies):
            print(f"  - {issue}")
            if i >= 4: # Show first 5 inconsistencies only for brevity
                print("  ... (more inconsistencies hidden)")
                break
    else:
        print("\nAll processed lines conform to the 'Word\\tLemma\\tPOS_Tag' format within documents.")

    if example_lines:
        print("\n--- Examples of Valid Lines (Word\\tLemma\\tPOS_Tag) ---")
        for example in example_lines:
            print(f"  {example}")
    else:
        print("\nNo valid example lines were found (corpus might be empty or entirely inconsistent).")

    print("\n--- Additional Notes ---")
    print("In this specific corpus format, an 'inconsistency' means a line that doesn't have exactly 3 tab-separated components (Word, Lemma, POS Tag).")
    print("Missing blank lines between documents or malformed <doc id> tags would also be inconsistencies.")

# --- Run the verification ---
verify_corpus_format("corpus_etiquetado.txt")


--- Verifying format of 'corpus_etiquetado.txt' ---

--- Summary for 'corpus_etiquetado.txt' ---
Total documents processed: 20

All processed lines conform to the 'Word\tLemma\tPOS_Tag' format within documents.

--- Examples of Valid Lines (Word\tLemma\tPOS_Tag) ---
  Doc 1, Line 3: 'Habla	hablar	VMIP3S0' (Valid)
  Doc 1, Line 4: 'con	con	SP' (Valid)
  Doc 1, Line 5: 'el	el	DA' (Valid)

--- Additional Notes ---
In this specific corpus format, an 'inconsistency' means a line that doesn't have exactly 3 tab-separated components (Word, Lemma, POS Tag).
Missing blank lines between documents or malformed <doc id> tags would also be inconsistencies.


# Parte 2: Extracción de características 
## Vector de características binario basado en la presencia de palabras del vocabulario
    en el contexto dado.

    + Args:
        contexto (list): Una lista de palabras que forman el contexto de la palabra ambigua.
                         Puede contener tuplas (palabra, tag) o solo palabras.
        vocabulario (list): Una lista de palabras que conforman el vocabulario de características.

    + Returns:
        list: Un vector binario donde cada elemento indica la presencia (1) o ausencia (0)
              de la palabra correspondiente del vocabulario en el contexto.
    

In [362]:
class Instance:
    def __init__(self, context, ambiguous_word_index):
        self.context = context
        self.ambiguous_word_index = ambiguous_word_index # Índice de la palabra ambigua en el contexto

# Ejemplo de contexto para 'hard'
# Asumimos que 'hard' es la palabra ambigua y está en alguna posición
# Para simplificar, aquí el contexto es una lista de palabras.
hard_instance_context = vocab

Extracción de características basada en las palabras vecinas

In [363]:
inst.context

[('clever', 'NNP'),
 ('white', 'NNP'),
 ('house', 'NNP'),
 ('``', '``'),
 ('spin', 'VB'),
 ('doctors', 'NNS'),
 ("''", "''"),
 ('are', 'VBP'),
 ('having', 'VBG'),
 ('a', 'DT'),
 ('hard', 'JJ'),
 ('time', 'NN'),
 ('helping', 'VBG'),
 ('president', 'NNP'),
 ('bush', 'NNP'),
 ('explain', 'VB'),
 ('away', 'RB'),
 ('the', 'DT'),
 ('economic', 'JJ'),
 ('bashing', 'NN'),
 ('that', 'IN'),
 ('low-and', 'JJ'),
 ('middle-income', 'JJ'),
 ('workers', 'NNS'),
 ('are', 'VBP'),
 ('taking', 'VBG'),
 ('these', 'DT'),
 ('days', 'NNS'),
 ('.', '.')]

In [364]:
vocab = ['time', 'would', 'get', 'work', 'find', 'make']
vocab = set(vocab) 
vocab# Convert to set for faster lookup

{'find', 'get', 'make', 'time', 'work', 'would'}

In [365]:
def extraer_caracteristicas_palabras_vecinas(contexto, vocabulario):

    vector_caracteristicas = []
    contexto_palabras_lower = []

    # Normalizar el contexto a solo palabras en minúsculas
    for item in contexto:
        word_to_add = ''
        if isinstance(item, tuple):
            word_to_add = item[0].lower()
        elif isinstance(item, str):
            word_to_add = item.lower()
        if word_to_add:
            contexto_palabras_lower.append(word_to_add)

    # Construir el vector de características
    for palabra_vocabulario in vocabulario:
        if palabra_vocabulario.lower() in contexto_palabras_lower:
            vector_caracteristicas.append(1)
        else:
            vector_caracteristicas.append(0)
            
    return vector_caracteristicas

In [366]:
# Contexto de la instancia de "hard"
# Simulamos un contexto donde "time" aparece y el resto no.
contexto_hard = vocab

# Extraer el vector de características
vector_caracteristicas_hard = extraer_caracteristicas_palabras_vecinas(contexto_hard, vocab)

print(f"Vocabulario: {vocab}")
print(f"Contexto de 'hard': {' '.join(contexto_hard)}")
print(f"Vector de características para 'hard': {vector_caracteristicas_hard}")



# Si el contexto_hard fuera el que describiste:
contexto_hard_solicitado = ["este", "es", "un", "ejemplo", "con", "time", "pero", "sin", "would", "get", "work", "find", "make"]
vector_caracteristicas_hard_solicitado = extraer_caracteristicas_palabras_vecinas(contexto_hard_solicitado, vocab)
print(f"Contexto de 'hard' (ejemplo solicitado): {' '.join(contexto_hard_solicitado)}")
print(f"Vector de características para 'hard' (ejemplo solicitado): {vector_caracteristicas_hard_solicitado}")

Vocabulario: {'get', 'make', 'time', 'would', 'find', 'work'}
Contexto de 'hard': get make time would find work
Vector de características para 'hard': [1, 1, 1, 1, 1, 1]
Contexto de 'hard' (ejemplo solicitado): este es un ejemplo con time pero sin would get work find make
Vector de características para 'hard' (ejemplo solicitado): [1, 1, 1, 1, 1, 1]


In [367]:
    # Contexto de la instancia de "hard"
# Simulamos un contexto donde "time" aparece y el resto no.
contexto_hard = ["este", "vector", "de", "características", "indica", "que", "en", "el", "contexto", "de", "la", "palabra",
                 "ambigua", "aparece", "la", "palabra", "time", "y", "no", "aparecen", "las", "palabras",
                 "would", "get", "work", "find", "y", "make"]

# Extraer el vector de características
vector_caracteristicas_hard = extraer_caracteristicas_palabras_vecinas(contexto_hard, vocab)

print(f"Vocabulario: {vocab}")
print(f"Contexto de 'hard': {' '.join(contexto_hard)}")
print(f"Vector de características para 'hard': {vector_caracteristicas_hard}")



# Si el contexto_hard fuera el que describiste:
contexto_hard_solicitado = ["este", "es", "un", "ejemplo", "con", "time", "pero", "sin", "would", "get", "work", "find", "make"]
vector_caracteristicas_hard_solicitado = extraer_caracteristicas_palabras_vecinas(contexto_hard_solicitado, vocab)
print(f"Contexto de 'hard' (ejemplo solicitado): {' '.join(contexto_hard_solicitado)}")
print(f"Vector de características para 'hard' (ejemplo solicitado): {vector_caracteristicas_hard_solicitado}")

Vocabulario: {'get', 'make', 'time', 'would', 'find', 'work'}
Contexto de 'hard': este vector de características indica que en el contexto de la palabra ambigua aparece la palabra time y no aparecen las palabras would get work find y make
Vector de características para 'hard': [1, 1, 1, 1, 1, 1]
Contexto de 'hard' (ejemplo solicitado): este es un ejemplo con time pero sin would get work find make
Vector de características para 'hard' (ejemplo solicitado): [1, 1, 1, 1, 1, 1]


# Construcción del vocabulario o bags of words.
"Este vector de características indica que en el contexto de la palabra
ambigua aparece la palabra «time» y no aparecen las palabras
«would», «get», «work», «find» y «make»."
Esto se traduciría a [1, 0, 0, 0, 0, 0] para el vocabulario dado.
Si en el contexto_hard las palabras "would", "get", "work", "find", "make" no estuvieran presentes,
el resultado sería exactamente [1, 0, 0, 0, 0, 0].
En el contexto_hard de ejemplo, las palabras "would", "get", "work", "find", "make" sí aparecen.
Por lo tanto, el vector resultante sería:
Palabra 'time': 1 (aparece)
Palabra 'would': 1 (aparece)
Palabra 'get': 1 (aparece)
Palabra 'work': 1 (aparece)
Palabra 'find': 1 (aparece)
Palabra 'make': 1 (aparece)
--> [1, 1, 1, 1, 1, 1]

In [387]:
# Asegúrate de haber descargado los recursos necesarios de NLTK
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt')
except nltk.downloader.DownloadError:
    nltk.download('punkt')


# Simulación de las instancias del corpus
# En un escenario real, 'hard_instances' y 'serve_instances'
# vendrían de tu corpus cargado.
class Instance:
    def __init__(self, context, ambiguous_word):
        self.context = context # Lista de palabras o (palabra, tag) tuplas
        self.ambiguous_word = ambiguous_word # La palabra ambigua asociada a esta instancia

        
# Ejemplos de instancias (simuladas para 'hard' y 'serve')
# Aquí usaremos solo strings en el contexto para simplificar, pero el código manejará tuplas también.
hard_instances = [
    Instance(["It", "was", "a", "hard", "time", "for", "everyone", "involved", "."], "hard"),
    Instance(["He", "worked", "hard", "to", "get", "the", "job", "done", "."], "hard"),
    Instance(["The", "decision", "was", "harder", "than", "I", "thought", "."], "hard"),
    Instance(["This", "is", "the", "hardest", "challenge", "I've", "faced", "."], "hard"),
    Instance(["You", "would", "find", "it", "hard", "to", "make", "a", "living", "."], "hard"),
    Instance(["It's", "hard", "work", "but", "it's", "rewarding", "."], "hard"),
    Instance(["They", "get", "through", "hard", "times", "."], "hard"),
    Instance(["It's", "hard", "to", "get", "enough", "sleep", "."], "hard"),
]

serve_instances = [
    Instance(["He", "will", "serve", "his", "country", "."], "serve"),
    Instance(["The", "waiter", "serves", "food", "."], "serve"),
    Instance(["She", "served", "in", "the", "military", "."], "serve"),
    Instance(["Serving", "the", "community", "is", "important", "."], "serve"),
]

# Recopilar todas las instancias en una sola lista para procesar todo el corpus
all_instances = hard_instances + serve_instances


Extracción de características basada en características de colocación


In [388]:
# Stopwords y otros signos a excluir
OTHER_WORDS = ["''","'d","'ll","'m","'re","'s","'t","'ve",'--','000','1','2','3','4','5','6','8','10','15','30','I','F','``',
               'also',"don'",'n','one','said','say','says','u','us']

# Combinar stopwords de NLTK, puntuación y OTHER_WORDS
STOPWORDS_SET = set(stopwords.words('english')).union(set(string.punctuation), set(OTHER_WORDS))

# Identificar las formas gramaticales de tus palabras ambiguas
# Aquí es donde debes añadir las formas que identificaste en tu laboratorio.

hard_forms = vocab
serve_forms = set()
for instance in serve_instances:
    for item in instance.context:
        word_to_check = ''
        if isinstance(item, tuple):
            word_to_check = item[0]
        elif isinstance(item, str):
            word_to_check = item
        if word_to_check and word_to_check.lower().startswith('serve'):
            serve_forms.add(word_to_check.lower())

# Combinar todas las palabras a excluir
WORDS_TO_EXCLUDE = STOPWORDS_SET.union(hard_forms).union(serve_forms)

print(f"Total de palabras a excluir: {len(WORDS_TO_EXCLUDE)}")
# print(f"Ejemplo de palabras a excluir: {list(WORDS_TO_EXCLUDE)[:20]}") # Descomenta para ver algunas

Total de palabras a excluir: 271


In [389]:
senseval.instances('hard.pos')[2737].context

[('``', '``'),
 ('it', 'PRP'),
 ("'s", 'VBZ'),
 ('a', 'DT'),
 ('very', 'RB'),
 ('interesting', 'JJ'),
 ('place', 'NN'),
 ('to', 'TO'),
 ('work', 'VB'),
 (',', ','),
 ('but', 'CC'),
 ('i', 'PRP'),
 ('can', 'MD'),
 ('see', 'VB'),
 ('why', 'WRB'),
 ('some', 'DT'),
 ('people', 'NNS'),
 ('have', 'VBP'),
 ('a', 'DT'),
 ('hard', 'JJ'),
 ('time', 'NN'),
 ('imagining', 'VBG'),
 ('what', 'WP'),
 ('it', 'PRP'),
 ("'s", 'VBZ'),
 ('like', 'IN'),
 (',', ','),
 ('"', '"'),
 ('says', 'VBZ'),
 ('nate', 'NNP'),
 ('gossett', 'NNP'),
 ('.', '.')]

Cómo construyo mi conjunto y que n utilizo? usando la librería ngrams

Extrae características contextuales (palabras anteriores y siguientes) de longitud 'n'
 alrededor de la palabra ambigua objetivo en una instancia de Senseval.
 Maneja condiciones de límite donde la palabra ambigua está al principio o al final de la oración.

 Argumentos:
     instancia (nltk.corpus.reader.senseval.SensevalInstance): La instancia a procesar.
    n (int): El número de palabras a considerar antes y después de la palabra objetivo.

 Retorna:
     dict: Un diccionario de características. Las claves serán como 'anterior(palabra1 palabra2)' o 'siguiente(palabra1 palabra2)'.

In [390]:
from nltk import ngrams # Import ngrams as suggested in the note 

def extract_contextual_features(instance, n=2):
 
    features = {}
    
    # Extract only the words from the context (list of (word, pos_tag) tuples)
    context_words = [word for word, _pos in instance.context]
    target_position = instance.position

    # --- Extract Previous Words ---
    # Calculate start index for previous words
    # It ensures we don't go below 0
    previous_start = max(0, target_position - n)
    previous_words = context_words[previous_start:target_position]

    if previous_words:
        # Join words to form a string for the feature key
        feature_key = f"previous({' '.join(previous_words)})"
        features[feature_key] = True
    else:
        # If there are no words before (e.g., target word is at the beginning)
        # You might choose to add a feature indicating lack of previous context,
        # or just omit the 'previous' feature for this instance.
        # For this example, we'll just not add the feature if no words exist.
        pass

    # --- Extract Next Words ---
    # Calculate end index for next words
    # It ensures we don't go beyond the end of the context
    next_end = min(len(context_words), target_position + n + 1)
    next_words = context_words[target_position + 1:next_end]

    if next_words:
        # Join words to form a string for the feature key
        feature_key = f"next({' '.join(next_words)})"
        features[feature_key] = True
    else:
        # If there are no words after (e.g., target word is at the end)
        pass

    return features

# --- Main script to demonstrate feature extraction ---

# 1. Ensure the 'senseval' corpus is downloaded
try:
    # Attempt to load a part of the corpus to check if it's available
    _ = senseval.instances('hard.pos')
    print("The 'senseval' corpus is already downloaded and accessible.")
except (OSError, LookupError):
    print("The 'senseval' corpus was not found. Downloading now...")
    nltk.download('senseval')
    print("Download complete!")
    from nltk.corpus import senseval # Re-import after download

# 2. Load instances for 'hard.pos' to test
try:
    hard_instances = senseval.instances('hard.pos')
    print(f"\nLoaded {len(hard_instances)} instances from 'hard.pos'.")
except OSError as e:
    print(f"Error loading instances from 'hard.pos': {e}")
    print("Please ensure your NLTK data path is correctly configured or restart your environment.")
    exit()

# 3. Process and display features for a few instances, including boundary cases
print("\n--- Feature Extraction Examples (n=2) ---")
processed_count = 0

for i, instance in enumerate(hard_instances):
    if processed_count >= 5: # Limit examples for brevity
        break

    features = extract_contextual_features(instance, n=2)

    # Print relevant instance info for context
    context_words_only = [word for word, _pos in instance.context]
    target_word_display = instance.word
    
    # Highlight the target word in the context for better visualization
    # Handle cases where position might be invalid for some reason, though unlikely in Senseval
    if 0 <= instance.position < len(context_words_only):
        context_display = context_words_only[:instance.position] + [f'**{target_word_display}**'] + context_words_only[instance.position+1:]
    else:
        context_display = context_words_only + [f'(Target word "{target_word_display}" at invalid position {instance.position})']

    print(f"\nInstance {i+1}:")
    print(f"  Target Word: '{instance.word}'")
    print(f"  Context: {' '.join(context_display)}")
    print(f"  Position: {instance.position}")
    print(f"  Extracted Features: {features}")
    processed_count += 1

print("\n--- Example of boundary case simulation (target at start/end) ---")

# Simulate an instance where 'hard' is at the very beginning
simulated_instance_start = nltk.corpus.reader.senseval.SensevalInstance(
    word='hard',
    senses=('hard%a%01',),
    context=[('hard', 'JJ'), ('work', 'NN'), ('pays', 'VBZ'), ('off', 'RP')],
    position=0
)
features_start = extract_contextual_features(simulated_instance_start, n=2)
print(f"\nSimulated (Start): Context: {' '.join([w for w,_ in simulated_instance_start.context])}, Position: {simulated_instance_start.position}")
print(f"  Features: {features_start}")
# Expected: {'next(work pays)': True}

# Simulate an instance where 'hard' is at the very end
simulated_instance_end = nltk.corpus.reader.senseval.SensevalInstance(
    word='hard',
    senses=('hard%a%01',),
    context=[('It', 'PRP'), ('was', 'VBD'), ('very', 'RB'), ('hard', 'JJ')],
    position=3
)
features_end = extract_contextual_features(simulated_instance_end, n=2)
print(f"\nSimulated (End): Context: {' '.join([w for w,_ in simulated_instance_end.context])}, Position: {simulated_instance_end.position}")
print(f"  Features: {features_end}")
# Expected: {'previous(was very)': True}

# Simulate an instance with less than n words for previous/next
simulated_instance_short_context = nltk.corpus.reader.senseval.SensevalInstance(
    word='hard',
    senses=('hard%a%01',),
    context=[('A', 'DT'), ('hard', 'JJ'), ('choice', 'NN')],
    position=1
)
features_short = extract_contextual_features(simulated_instance_short_context, n=2)
print(f"\nSimulated (Short Context): Context: {' '.join([w for w,_ in simulated_instance_short_context.context])}, Position: {simulated_instance_short_context.position}")
print(f"  Features: {features_short}")
# Expected: {'previous(A)': True, 'next(choice)': True}


The 'senseval' corpus is already downloaded and accessible.

Loaded 4333 instances from 'hard.pos'.

--- Feature Extraction Examples (n=2) ---

Instance 1:
  Target Word: 'hard-a'
  Context: `` he may lose all popular support , but someone has to kill him to defeat him and that 's **hard-a** to do . ''
  Position: 20
  Extracted Features: {"previous(that 's)": True, 'next(to do)': True}

Instance 2:
  Target Word: 'hard-a'
  Context: clever white house `` spin doctors '' are having a **hard-a** time helping president bush explain away the economic bashing that low-and middle-income workers are taking these days .
  Position: 10
  Extracted Features: {'previous(having a)': True, 'next(time helping)': True}

Instance 3:
  Target Word: 'hard-a'
  Context: i find it **hard-a** to believe that the sacramento river will ever be quite the same , although i certainly wish that i 'm wrong .
  Position: 3
  Extracted Features: {'previous(find it)': True, 'next(to believe)': True}

Instance 4:
  

In [391]:
all_words = []
for instance in all_instances:
    for item in instance.context:
        word = ''
        if isinstance(item, tuple):
            word = item[0]
        elif isinstance(item, str):
            word = item
        
        # Procesar solo si la palabra no está vacía y no es una forma gramatical de la palabra ambigua (inicialmente)
        # La exclusión de otras palabras se hará después de FreqDist
        if word:
            all_words.append(word.lower())

# Calcular la distribución de frecuencias de todas las palabras
freq_dist = FreqDist(all_words)

print(f"\nDistribución de frecuencias inicial (top 10): {freq_dist.most_common(10)}")


Distribución de frecuencias inicial (top 10): [('.', 12), ('hard', 6), ('the', 6), ('to', 3), ('get', 3), ("it's", 3), ('it', 2), ('was', 2), ('a', 2), ('he', 2)]


In [392]:
def construir_vocabulario_frecuente(freq_dist, words_to_exclude, m):
  
    filtered_vocab = []
    
    # Itera sobre las palabras más comunes en orden descendente de frecuencia
    for word, frequency in freq_dist.most_common():
        if word not in words_to_exclude:
            filtered_vocab.append(word)
        
        # Detener una vez que hemos alcanzado el tamaño de vocabulario deseado (m)
        if len(filtered_vocab) >= m:
            break
            
    return filtered_vocab

# Definir el tamaño del vocabulario deseado (m)
m_palabras = 6 # Para el ejemplo de 'hard'

# Construir el vocabulario final
vocabulario_final = construir_vocabulario_frecuente(freq_dist, WORDS_TO_EXCLUDE, m_palabras)

print(f"\n--- Vocabulario Construido ---")
print(f"Número de palabras más frecuentes (m): {m_palabras}")
print(f"Vocabulario final: {vocabulario_final}")

# Comprobación con el ejemplo de 'hard'
# Si las instancias de 'hard' contienen las palabras 'time', 'would', 'get', 'work', 'find', 'make'
# y estas son las 6 más frecuentes después del filtrado, entonces coincidirán.
# Con mis datos simulados, el resultado será diferente al ejemplo dado inicialmente,
# ya que el ejemplo de 'hard' tenía un vocabulario predefinido.
# Para que coincida con ['time', 'would', 'get', 'work', 'find', 'make'],
# esas palabras deberían ser las 6 más frecuentes en TU corpus real después de la limpieza.


--- Vocabulario Construido ---
Número de palabras más frecuentes (m): 6
Vocabulario final: ['hard', 'everyone', 'involved', 'worked', 'job', 'done']


Conjunto combinado de stopwords de NLTK (inglés), puntuación y tus palabras adicionales
 Nota: NLTK stopwords por defecto son en inglés. Si tu corpus está en español,
 necesitarás cambiar 'english' a 'spanish' en 'stopwords.words()'.
 Para este ejemplo, mantendré 'english' como en tu referencia, pero tenlo en cuenta.

Construye un vocabulario de las 'm' palabras más frecuentes del corpus,
    excluyendo puntuación, stopwords y formas gramaticales de las palabras ambiguas.

    Args:
        instances (list): Lista de objetos Instance, cada uno con un contexto.
        m (int): El número de palabras más frecuentes a incluir en el vocabulario.
        palabras_ambiguas_forms (dict): Un diccionario donde la clave es la palabra ambigua
                                        y el valor es un conjunto de sus formas gramaticales
                                        a excluir (ej: {'hard': {'hard', 'harder', 'hardest'}}).

    Returns:
        list: El vocabulario construido (lista de palabras).

In [393]:
# --- Definición de Stop Words y Palabras Adicionales a Eliminar ---
# Tu conjunto de palabras a eliminar
OTHER_WORDS = ["''", "'d", "'ll", "'m", "'re", "'s", "'t", "'ve", '--', '000', '1', '2', '3', '4', '5', '6', '8', '10', '15', '30', 'I', 'F', '``',
               'also', "don'", 'n', 'one', 'said', 'say', 'says', 'u', 'us']


STOPWORDS_SET = set(stopwords.words('english')).union(set(string.punctuation), set(OTHER_WORDS))

# Clase dummy para simular las instancias del corpus
class Instance:
    def __init__(self, context, ambiguous_word):
        # context puede ser una lista de palabras o de tuplas (palabra, tag)
        self.context = context
        self.ambiguous_word = ambiguous_word

def construir_vocabulario(instances, m, palabras_ambiguas_forms):

    all_words = []
    
    # Recopilar todas las palabras de todos los contextos
    for instance in instances:
        for item in instance.context:
            word = ''
            if isinstance(item, tuple):
                word = item[0] # Extraer la palabra de la tupla (si hay POS tags)
            elif isinstance(item, str):
                word = item # La palabra ya es un string
            
            # Normalizar a minúsculas
            word_lower = word.lower()
            all_words.append(word_lower)
            
    # Usar FreqDist para calcular las frecuencias
    fdist = nltk.FreqDist(all_words)
    
    # Filtrar palabras: eliminar puntuación, stopwords y formas de palabras ambiguas
    # Creamos un conjunto de todas las palabras a excluir
    words_to_exclude = set()
    words_to_exclude.update(STOPWORDS_SET)
    
    for amb_word_forms in palabras_ambiguas_forms.values():
        words_to_exclude.update(amb_word_forms)

    filtered_words_freq = Counter()
    for word, freq in fdist.items():
        if word not in words_to_exclude:
            filtered_words_freq[word] = freq
            
    # Obtener las 'm' palabras más frecuentes
    vocabulario = [word for word, freq in filtered_words_freq.most_common(m)]
    
    return vocabulario

# --- Ejemplo de Uso ---

# 1. Simular datos del corpus (tus instancias de "hard" y "serve")
# Debes reemplazar esto con tus datos reales (hard_instances, serve_instances)
# Ejemplo de contextos
corpus_instances = [
    Instance(context=["this", "is", "a", "hard", "time", "for", "everyone", "and", "we", "will", "get", "through", "it"], ambiguous_word="hard"),
    Instance(context=["it's", "hard", "to", "find", "a", "job", "these", "days"], ambiguous_word="hard"),
    Instance(context=["the", "machine", "will", "serve", "you", "well"], ambiguous_word="serve"),
    Instance(context=["they", "serve", "delicious", "food", "at", "that", "workplace"], ambiguous_word="serve"),
    Instance(context=["a", "time", "to", "work", "harder", "than", "ever"], ambiguous_word="hard"),
    Instance(context=["he", "would", "like", "to", "make", "a", "difference"], ambiguous_word="would"), # Una instancia extra para 'would'
    Instance(context=["you", "should", "get", "up", "and", "make", "it", "happen"], ambiguous_word="get"), # Instancia para 'get', 'make'
    Instance(context=["find", "your", "own", "way", "to", "work", "it", "out"], ambiguous_word="find") # Instancia para 'find', 'work'
]

# 2. Definir las formas gramaticales de las palabras ambiguas
# Aquí es donde añadirías las formas que identificaste en la Parte 1
palabras_ambiguas_forms = {
    'hard': {'hard', 'harder', 'hardest'},
    'serve': {'serve', 'serves', 'served', 'serving'} # Ejemplos para 'serve'
}

# 3. Construir el vocabulario con un tamaño 'm' (por ejemplo, m=6)
m_size = 6
vocab_construido = construir_vocabulario(corpus_instances, m_size, palabras_ambiguas_forms)

print(f"Número de palabras en el vocabulario deseado (m): {m_size}")
print(f"Vocabulario construido: {vocab_construido}")


Número de palabras en el vocabulario deseado (m): 6
Vocabulario construido: ['time', 'get', 'find', 'work', 'make', 'everyone']


El código construye un vocabulario a partir de textos, filtrando palabras irrelevantes. Utiliza nltk para calcular la frecuencia de palabras y un conjunto de stopwords (que deben ser en español), puntuación y formas gramaticales de palabras ambiguas para excluirlas. La función construir_vocabulario procesa las instancias, recopila todas las palabras, calcula sus frecuencias y luego filtra las no deseadas. Finalmente, selecciona las m palabras más comunes y relevantes para formar el vocabulario. Este proceso es crucial para la creación de vectores de características en el procesamiento de lenguaje natural.


Construye un vocabulario de las 'm' palabras más frecuentes del corpus,
    excluyendo puntuación, stopwords y formas gramaticales de las palabras ambiguas.
    (Esta función es la misma que la proporcionada anteriormente)


Como se observa se generan dos instancias similares y se realiza la prueba para obtener el procedimiento finalizado


In [394]:

# Tu conjunto de palabras a eliminar (igual al anterior)
OTHER_WORDS = ["''", "'d", "'ll", "'m", "'re", "'s", "'t", "'ve", '--', '000', '1', '2', '3', '4', '5', '6', '8', '10', '15', '30', 'I', 'F', '``',
               'also', "don'", 'n', 'one', 'said', 'say', 'says', 'u', 'us']

# Conjunto combinado de stopwords de NLTK (inglés), puntuación y tus palabras adicionales
STOPWORDS_SET = set(stopwords.words('english')).union(set(string.punctuation), set(OTHER_WORDS))

def construir_vocabulario(instances, m, palabras_ambiguas_forms):
   
    all_words = []
    for instance in instances:
        for item in instance.context:
            word = ''
            if isinstance(item, tuple):
                word = item[0]
            elif isinstance(item, str):
                word = item
            word_lower = word.lower()
            all_words.append(word_lower)
            
    fdist = nltk.FreqDist(all_words)
    
    words_to_exclude = set()
    words_to_exclude.update(STOPWORDS_SET)
    
    for amb_word_forms in palabras_ambiguas_forms.values():
        words_to_exclude.update(amb_word_forms)

    filtered_words_freq = Counter()
    for word, freq in fdist.items():
        if word not in words_to_exclude:
            filtered_words_freq[word] = freq
            
    vocabulario = [word for word, freq in filtered_words_freq.most_common(m)]
    
    return vocabulario

# --- Nueva función para extraer el vector de características como diccionario ---

def extraer_caracteristicas_diccionario(instance_context, vocabulario):
    vector_caracteristicas = {}
    contexto_palabras_lower = []

    # Normalizar el contexto a solo palabras en minúsculas
    for item in instance_context:
        word_to_add = ''
        if isinstance(item, tuple):
            word_to_add = item[0]
        elif isinstance(item, str):
            word_to_add = item
        
        if word_to_add:
            contexto_palabras_lower.append(word_to_add.lower())

    # Construir el diccionario de características
    for palabra_vocabulario in vocabulario:
        clave = f'contains({palabra_vocabulario})'
        if palabra_vocabulario.lower() in contexto_palabras_lower:
            vector_caracteristicas[clave] = True
        else:
            vector_caracteristicas[clave] = False
            
    return vector_caracteristicas

# --- Ejemplo Completo ---

# 1. Simular datos del corpus (usando los mismos datos de ejemplo que antes)
corpus_instances = [
    Instance(context=["this", "is", "a", "hard", "time", "for", "everyone", "and", "we", "will", "get", "through", "it"], ambiguous_word="hard"),
    Instance(context=["it's", "hard", "to", "find", "a", "job", "these", "days"], ambiguous_word="hard"),
    Instance(context=["the", "machine", "will", "serve", "you", "well"], ambiguous_word="serve"),
    Instance(context=["they", "serve", "delicious", "food", "at", "that", "workplace"], ambiguous_word="serve"),
    Instance(context=["a", "time", "to", "work", "harder", "than", "ever"], ambiguous_word="hard"),
    Instance(context=["he", "would", "like", "to", "make", "a", "difference"], ambiguous_word="would"),
    Instance(context=["you", "should", "get", "up", "and", "make", "it", "happen"], ambiguous_word="get"),
    Instance(context=["find", "your", "own", "way", "to", "work", "it", "out"], ambiguous_word="find")
]

# 2. Definir las formas gramaticales de las palabras ambiguas
palabras_ambiguas_forms = {
    'hard': {'hard', 'harder', 'hardest'},
    'serve': {'serve', 'serves', 'served', 'serving'}
}

# 3. Construir el vocabulario (ejemplo con m=6)
m_size = 6
vocabulario_generado = construir_vocabulario(corpus_instances, m_size, palabras_ambiguas_forms)

print(f"Vocabulario generado para m={m_size}: {vocabulario_generado}")
print("---")

# 4. Mostrar el vector de características para una instancia del corpus
# Tomemos la primera instancia como ejemplo:
# "this is a hard time for everyone and we will get through it"

instancia_ejemplo = corpus_instances[0]
print(f"Instancia de ejemplo - Palabra ambigua: '{instancia_ejemplo.ambiguous_word}'")
print(f"Contexto (oración completa): {' '.join(instancia_ejemplo.context)}")

# Extraer el vector de características para esta instancia
vector_caracteristicas_instancia = extraer_caracteristicas_diccionario(instancia_ejemplo.context, vocabulario_generado)

print(f"Vector de características para la instancia (diccionario):\n{vector_caracteristicas_instancia}")

print("---")

# Para el ejemplo solicitado: {'contains(time)': True, 'contains(would)': False, ...}
# Usaremos un vocabulario fijo para ilustrar ese caso específico.
vocabulario_fijo_ejemplo = ['time', 'would', 'get', 'work', 'find', 'make']
contexto_ejemplo_solicitado = ["este", "vector", "de", "características", "indica", "que", "en", "el", "contexto", "de", "la", "palabra",
                               "ambigua", "aparece", "la", "palabra", "time", "y", "no", "aparecen", "las", "palabras",
                               "would", "get", "work", "find", "y", "make"] # Aquí, todas aparecen en el ejemplo de texto.

print("--- Ejemplo específico del enunciado ---")
print(f"Vocabulario de ejemplo del enunciado: {vocabulario_fijo_ejemplo}")
print(f"Contexto del enunciado: {' '.join(contexto_ejemplo_solicitado)}")

# Si el contexto del ejemplo del enunciado solo contuviera 'time' y NO el resto:
contexto_solo_time = ["este", "vector", "de", "características", "indica", "que", "en", "el", "contexto", "de", "la", "palabra",
                      "ambigua", "aparece", "la", "palabra", "time"]

vector_caracteristicas_solicitado = extraer_caracteristicas_diccionario(contexto_solo_time, vocabulario_fijo_ejemplo)
print(f"Vector de características (si solo 'time' aparece en el contexto): {vector_caracteristicas_solicitado}")

Vocabulario generado para m=6: ['time', 'get', 'find', 'work', 'make', 'everyone']
---
Instancia de ejemplo - Palabra ambigua: 'hard'
Contexto (oración completa): this is a hard time for everyone and we will get through it
Vector de características para la instancia (diccionario):
{'contains(time)': True, 'contains(get)': True, 'contains(find)': False, 'contains(work)': False, 'contains(make)': False, 'contains(everyone)': True}
---
--- Ejemplo específico del enunciado ---
Vocabulario de ejemplo del enunciado: ['time', 'would', 'get', 'work', 'find', 'make']
Contexto del enunciado: este vector de características indica que en el contexto de la palabra ambigua aparece la palabra time y no aparecen las palabras would get work find y make
Vector de características (si solo 'time' aparece en el contexto): {'contains(time)': True, 'contains(would)': False, 'contains(get)': False, 'contains(work)': False, 'contains(find)': False, 'contains(make)': False}


In [395]:
def calcular_probabilidades_emision(corpus):
    emision = {}
    total_etiquetas = {}
    for token, etiqueta in corpus:
        if etiqueta not in emision:
            emision[etiqueta] = {}
            total_etiquetas[etiqueta] = 0
        emision[etiqueta][token] = emision[etiqueta].get(token, 0) + 1
        total_etiquetas[etiqueta] += 1
    for etiqueta in emision:
        for token in emision[etiqueta]:
            emision[etiqueta][token] /= total_etiquetas[etiqueta]
    return emision

def calcular_probabilidades_transicion(corpus):
    transicion = {}
    total_transiciones = {}
    anterior = None
    for _, etiqueta in corpus:
        if anterior is not None:
            if anterior not in transicion:
                transicion[anterior] = {}
                total_transiciones[anterior] = 0
            transicion[anterior][etiqueta] = transicion[anterior].get(etiqueta, 0) + 1
            total_transiciones[anterior] += 1
        anterior = etiqueta
    for etiqueta in transicion:
        for siguiente in transicion[etiqueta]:
            transicion[etiqueta][siguiente] /= total_transiciones[etiqueta]
    return transicion

def calcular_probabilidades_iniciales(corpus):
    iniciales = {}
    total = 0
    if corpus:
        iniciales[corpus[0][1]] = 1
        total = 1
    for etiqueta in iniciales:
        iniciales[etiqueta] /= total
    return iniciales

emision = calcular_probabilidades_emision(corpus)
transicion = calcular_probabilidades_transicion(corpus)
estado_inicial = calcular_probabilidades_iniciales(corpus)

In [396]:
def viterbi(frase, estados, transicion, emision, estado_inicial):
    V = [{}]
    path = {}

    for estado in estados:
        V[0][estado] = estado_inicial.get(estado, 1e-6) * emision.get(estado, {}).get(frase[0], 1e-6)
        path[estado] = [estado]

    for t in range(1, len(frase)):
        V.append({})
        new_path = {}
        for y in estados:
            (prob, estado_max) = max(
                [(V[t-1][y0] * transicion.get(y0, {}).get(y, 1e-6) * emision.get(y, {}).get(frase[t], 1e-6), y0)
                 for y0 in estados], key=lambda x: x[0])
            V[t][y] = prob
            new_path[y] = path[estado_max] + [y]
        path = new_path

    n = len(frase) - 1
    (prob, estado_max) = max([(V[n][y], y) for y in estados], key=lambda x: x[0])
    return (prob, path[estado_max])

In [397]:
frase_prueba = ['Habla', 'con', 'el', 'enfermo', 'grave', 'de', 'trasplantes', '.']
estados = list(set([etiqueta for _, etiqueta in corpus]))

probabilidad, ruta = viterbi(frase_prueba, estados, transicion, emision, estado_inicial)
print("Frase:", frase_prueba)
print("Ruta más probable:", ruta)
print("Probabilidad total:", probabilidad)

Frase: ['Habla', 'con', 'el', 'enfermo', 'grave', 'de', 'trasplantes', '.']
Ruta más probable: ['VMIP3S0', 'SP', 'DA', 'NCMS000', 'AQ0CS00', 'SP', 'NCMN000', 'Fp']
Probabilidad total: 1.8533917043090838e-08


In [398]:
import pandas as pd

# (Assuming 'emision', 'transicion', 'frase_prueba', and 'ruta' are defined as above or loaded elsewhere)

# Create DataFrame for Emisión (Emission Probabilities)
# Rows will be words, columns will be tags
df_emision = pd.DataFrame.from_dict(emision, orient='index').fillna(0)
print("DataFrame de Emisión:")
print(df_emision)
print("\n" + "="*50 + "\n")


DataFrame de Emisión:
             Habla     habla     corre     canta    brilla    avanza  \
VMIP3S0   0.142857  0.142857  0.142857  0.142857  0.142857  0.142857   
SP        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
DA        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCMS000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
AQ0CS00   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCMN000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
Fp        0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NCFS000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VAIP3S0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VMP00SF   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
SP+DA     0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
NP00000   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
VSIP3S0   0.000000  0.000000  0.000000  0.

In [399]:
# Create DataFrame for Transición (Transition Probabilities)
# Rows will be current tags, columns will be next tags
df_transicion = pd.DataFrame.from_dict(transicion, orient='index').fillna(0)
print("DataFrame de Transición:")
print(df_transicion)
print("\n" + "="*50 + "\n")


DataFrame de Transición:
                SP        DI        RG   NCMP000        DA   NCMN000  \
VMIP3S0   0.428571  0.142857  0.285714  0.142857  0.000000  0.000000   
NCMS000   0.105263  0.052632  0.000000  0.000000  0.000000  0.000000   
AQ0CS00   0.142857  0.000000  0.000000  0.000000  0.000000  0.000000   
VMIP3P0   0.333333  0.000000  0.000000  0.000000  0.000000  0.000000   
RG        0.750000  0.000000  0.000000  0.000000  0.000000  0.000000   
VIIIS3S   0.500000  0.500000  0.000000  0.000000  0.000000  0.000000   
Fp        0.000000  0.157895  0.052632  0.000000  0.315789  0.000000   
VSIP3S0   0.000000  0.333333  0.000000  0.000000  0.000000  0.000000   
VMIS3S0   0.000000  0.000000  0.500000  0.000000  0.500000  0.000000   
AQ0MP00   0.000000  0.000000  0.000000  0.500000  0.000000  0.000000   
SP        0.000000  0.000000  0.000000  0.000000  0.636364  0.181818   
VIIIS0S   0.000000  0.000000  0.000000  0.000000  1.000000  0.000000   
DA        0.000000  0.000000  0.000000 

In [400]:

# Create DataFrame for Viterbi result
# Shows the input phrase words and their predicted tags
df_viterbi = pd.DataFrame({
    "Palabra": frase_prueba,
    "Etiqueta": ruta
})
print("DataFrame Viterbi (Palabra y Etiqueta):")
print(df_viterbi)


DataFrame Viterbi (Palabra y Etiqueta):
       Palabra Etiqueta
0        Habla  VMIP3S0
1          con       SP
2           el       DA
3      enfermo  NCMS000
4        grave  AQ0CS00
5           de       SP
6  trasplantes  NCMN000
7            .       Fp


In [401]:
df_emision.to_excel("tabla_emision.xlsx", engine='openpyxl')
df_transicion.to_excel("tabla_transicion.xlsx", engine='openpyxl')
df_viterbi.to_excel("ruta_viterbi.xlsx", engine='openpyxl')

from IPython.display import FileLink, display
display(FileLink("tabla_emision.xlsx"))
display(FileLink("tabla_transicion.xlsx"))
display(FileLink("ruta_viterbi.xlsx"))

i:\unirIA\procesamientoLenguaje\senseval\tabla_emision.xlsx

i:\unirIA\procesamientoLenguaje\senseval\tabla_transicion.xlsx

i:\unirIA\procesamientoLenguaje\senseval\ruta_viterbi.xlsx